In [7]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    objects = []
    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        polygon = []
        for point in obj.find('polygon').findall('*'):
            if point.tag.startswith('x'):
                x = float(point.text)
            elif point.tag.startswith('y'):
                y = float(point.text)
                polygon.append((x, y))
        objects.append((obj_name, polygon))

    return width, height, objects


def create_mask(width, height, objects):
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    for _, polygon in objects:
        draw.polygon(polygon, fill=255)
    return mask

def main(xml_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    xml_files = [f for f in os.listdir(xml_folder) if f.endswith('.xml')]
    for xml_file in xml_files:
        xml_path = os.path.join(xml_folder, xml_file)
        image_id = os.path.splitext(xml_file)[0]
        width, height, objects = parse_annotation(xml_path)
        mask = create_mask(width, height, objects)
        mask_file = os.path.join(output_folder, image_id + '.png')
        mask.save(mask_file)

if __name__ == '__main__':
    xml_folder = r'I:\Dataset\sugarcanev3.v1i.voc\xml'  # Note the 'r' prefix to indicate a raw string
    output_folder = r'I:\Dataset\sugarcanev3.v1i.voc\PedMasks'  # Note the 'r' prefix to indicate a raw string
    main(xml_folder, output_folder)
